In [1]:
import subprocess
import pandas as pd
import re
import json
import seaborn as sns 
print(sns.__version__) # should be 0.9

import matplotlib.pyplot as plt
sns.set(style="whitegrid")

from IPython.display import display, HTML


0.9.0


In [9]:
# Compile files
subprocess.run(["chmod", "+x" "compile.sh"])
subprocess.run(["chmod", "+x" "run.sh"])
subprocess.run(["./compile.sh"])


# def runs():
#     ps = ("uniform", "deterministic_fitness", "ranking_parent", "stochastic_parent", "stochastic_parent_no_repitition", "tournament_parent")
#     rec = ("uniform", "simple_arithmetic")
    
#     for p in ps:
#         for re in rec:
#             for i in range(2,10):
#                 yield (p, re, i)


subprocess.run(["rm", "./log/bent_cigar.txt", "./log/katsura.txt", "./log/chaffers.txt"])

# for (parentselection, recombination, parents) in runs():
#     subprocess.run(["./run.sh", '"{}"', '"{}"', '"{}"'.format(parentselection, recombination, parents)])
subprocess.run(["./run.sh", parentselection, recombination, str(parents)])


KeyboardInterrupt: 

In [56]:
### For some reason the below only works for me with SphereEvaluation, not BentCigarFunction... Very annoying
### So just run the command and pipe it to a text file and read that out...

# result = subprocess.run(['java', '-jar', 'testrun.jar', '-evaluation=BentCigarFunction', '-seed=1', '-submission=player50'], stdout=subprocess.PIPE)
# output = result.stdout.decode('utf-8')
# print(result, result.stderr, result.stdout)
# print(output)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [10]:
import os


def create_run_df(run_output, run_config):
    df = pd.DataFrame()
    generation_nr = 0
    matches = re.finditer("Statistic({.+})", run_output)
    for m in matches:
        json_str = m[1]
        obj = pd.read_json(json_str, orient='column', typ='series')
        obj.at['run_config'] = run_config
        obj.at['generation_nr'] = generation_nr

        df = df.append(obj, ignore_index=True)
        generation_nr += 1
        
    return df

def create_all_runs_dfs(file_path):
    f = open(file_path, "r")
    output = f.read()
    run = 0
    print("Processing file: {}".format(file_path))
    for m in re.finditer("---Run ({.+})", output):
        print("Processing run {}".format(run))
#         print(m.start(), m.end())

        configuration_json = m.groups(1)[0]
        end_m = re.search("---Finished run---", output[m.start():])

#         print(end_m.start())
        if end_m is None:
            return

        run_output = output[m.end():end_m.end() + m.end()]

        df = create_run_df(run_output, configuration_json)

        run += 1
        yield df

# dfs = create_all_runs_dfs("bent_cigar.txt")


all_dfs = {'cigar': {}, 'schaffer': {}, 'katsuura': {}}
total_dfs = {'cigar': None, 'schaffer': None, 'katsuura': None}
runs = (('cigar', "./log/bent_cigar.txt",), ('katsuura', "./log/katsura.txt",), ('schaffer', "./log/chaffers.txt",))

for fn_key, file in runs:
    dfs = create_all_runs_dfs(file)
    df = next(dfs)
    all_dfs[fn_key] = {df['run_config'][0] : df}
    for next_df in dfs:
        config = next_df['run_config'][0]
        all_dfs[fn_key][config] = next_df
        total_dfs[fn_key] = df.append(next_df)
    
    
    


Processing file: ./log/bent_cigar.txt
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing run 15
Processing run 16
Processing run 17
Processing run 18
Processing run 19
Processing run 20
Processing run 21
Processing run 22
Processing run 23
Processing run 24
Processing run 25
Processing run 26
Processing run 27
Processing run 28
Processing run 29
Processing run 30
Processing run 31
Processing run 32
Processing run 33
Processing run 34
Processing run 35
Processing run 36
Processing run 37
Processing run 38
Processing run 39
Processing run 40
Processing run 41
Processing run 42
Processing run 43
Processing run 44
Processing run 45
Processing run 46
Processing run 47
Processing run 48
Processing run 49
Processing run 50
Processing run 51
Processing run 52
Processing run 53


In [11]:
# Mean max for different functions:
for fn, total_df in total_dfs.items():
    print(fn)
    mean = total_df.groupby(['run_config', 'generation_nr']).mean()
    max_idx = mean['maxFitness'].idxmax()
    
    display(mean.loc[max_idx])

cigar


averageAge                16.110000
averageFitness             9.635822
averageOfstdevGenomes      0.053297
maxAge                    99.000000
maxFitness                 9.778287
minAge                     0.000000
minFitness                 9.566117
populationSize           100.000000
stdevAge                  16.308829
stdevFitness               0.057865
Name: ({"parentScheme": uniform, "recombinationScheme": uniform, "nr_of_parents": 2 }, 316.0), dtype: float64

schaffer


averageAge                45.850000
averageFitness             5.293206
averageOfstdevGenomes      0.136228
maxAge                   222.000000
maxFitness                 6.764272
minAge                     0.000000
minFitness                 4.975411
populationSize           100.000000
stdevAge                  41.634931
stdevFitness               0.324043
Name: ({"parentScheme": uniform, "recombinationScheme": uniform, "nr_of_parents": 2 }, 420.0), dtype: float64

katsuura


averageAge                 8.650000
averageFitness             0.007815
averageOfstdevGenomes      0.063456
maxAge                    17.000000
maxFitness                 0.073716
minAge                     0.000000
minFitness                 0.007149
populationSize           100.000000
stdevAge                   5.396990
stdevFitness               0.006623
Name: ({"parentScheme": uniform, "recombinationScheme": uniform, "nr_of_parents": 2 }, 6962.0), dtype: float64

In [13]:
# print(all_dfs.keys())
for key, dfs in all_dfs.items():
    for config, df in dfs.items():
        print(key)
        print(config)
        df.head()
    
        display(HTML(df.tail(5).to_html()))

cigar
{"parentScheme": uniform, "recombinationScheme": uniform, "nr_of_parents": 2 }


,averageAge,averageFitness,averageOfstdevGenomes,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness,stdevGenomes
325,16.42,9.675190,0.043487,325.0,80.0,9.778287,0.0,9.598461,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",16.170455,0.051998,"[1.332267629550187e-15, 0.004894537645444, 0.1..."
326,15.90,9.679944,0.041288,326.0,81.0,9.778287,0.0,9.609155,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",15.012328,0.051636,"[1.332267629550187e-15, 0.0048856626832090005,..."
327,15.02,9.684274,0.040012,327.0,62.0,9.778287,0.0,9.617202,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",13.385051,0.050057,"[1.332267629550187e-15, 0.004874793564778, 0.1..."
328,14.99,9.690041,0.037629,328.0,63.0,9.778287,0.0,9.622020,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",13.101523,0.050623,"[1.332267629550187e-15, 0.004830074301333, 0.1..."
329,15.30,9.692738,0.035153,329.0,64.0,9.778287,0.0,9.624869,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",13.166245,0.048834,"[1.332267629550187e-15, 0.004741404572185, 0.0..."


cigar
{"parentScheme": uniform, "recombinationScheme": simple_arithmetic, "nr_of_parents": 2 }


,averageAge,averageFitness,averageOfstdevGenomes,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness,stdevGenomes
325,16.36,9.396569,0.001471,325.0,95.0,9.410287,0.0,9.392664,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",19.387377,0.003285,"[1.5543122344752194e-15, 9.769962616701378e-15..."
326,16.19,9.396837,0.001401,326.0,96.0,9.410287,0.0,9.393035,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",19.472902,0.003147,"[1.5543122344752194e-15, 9.769962616701378e-15..."
327,17.14,9.396880,0.001331,327.0,97.0,9.410287,0.0,9.393486,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",19.512570,0.003112,"[1.5543122344752194e-15, 9.769962616701378e-15..."
328,17.29,9.397080,0.001318,328.0,98.0,9.410287,0.0,9.393992,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",19.776903,0.003044,"[1.5543122344752194e-15, 9.769962616701378e-15..."
329,15.58,9.397307,0.001218,329.0,95.0,9.410287,0.0,9.394669,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",18.128530,0.002920,"[1.5543122344752194e-15, 9.769962616701378e-15..."


cigar
{"parentScheme": deterministic_fitness, "recombinationScheme": uniform, "nr_of_parents": 2 }


...

,averageAge,averageFitness,averageOfstdevGenomes,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness,stdevGenomes
6661,643.34,0.176498,1.277825e-15,6661.0,3573.0,0.176498,0.0,0.176498,100.0,"{""parentScheme"": ranking_parent, ""recombinatio...",1370.604496,1.683393e-14,"[4.440892098500626e-15, 3.885780586188048e-16,..."
6662,643.34,0.176498,1.277825e-15,6662.0,3574.0,0.176498,0.0,0.176498,100.0,"{""parentScheme"": ranking_parent, ""recombinatio...",1371.073056,1.683393e-14,"[4.440892098500626e-15, 3.885780586188048e-16,..."
6663,643.40,0.176498,1.277825e-15,6663.0,3575.0,0.176498,0.0,0.176498,100.0,"{""parentScheme"": ranking_parent, ""recombinatio...",1371.513369,1.683393e-14,"[4.440892098500626e-15, 3.885780586188048e-16,..."
6664,643.53,0.176498,1.277825e-15,6664.0,3576.0,0.176498,0.0,0.176498,100.0,"{""parentScheme"": ranking_parent, ""recombinatio...",1371.920978,1.683393e-14,"[4.440892098500626e-15, 3.885780586188048e-16,..."
6665,643.86,0.176498,1.277825e-15,6665.0,3577.0,0.176498,0.0,0.176498,100.0,"{""parentScheme"": ranking_parent, ""recombinatio...",1372.234893,1.683393e-14,"[4.440892098500626e-15, 3.885780586188048e-16,..."


katsuura
{"parentScheme": tournament_parent, "recombinationScheme": uniform, "nr_of_parents": 10 }


,averageAge,averageFitness,averageOfstdevGenomes,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness,stdevGenomes
6661,3598.93,0.005368,2.889525,6661.0,6589.0,0.092737,95.0,0.000336,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1832.306673,0.013572,"[3.243470388221461, 3.137221003257709, 2.80499..."
6662,3599.93,0.005368,2.889525,6662.0,6590.0,0.092737,96.0,0.000336,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1832.306673,0.013572,"[3.243470388221461, 3.137221003257709, 2.80499..."
6663,3600.93,0.005368,2.889525,6663.0,6591.0,0.092737,97.0,0.000336,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1832.306673,0.013572,"[3.243470388221461, 3.137221003257709, 2.80499..."
6664,3601.93,0.005368,2.889525,6664.0,6592.0,0.092737,98.0,0.000336,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1832.306673,0.013572,"[3.243470388221461, 3.137221003257709, 2.80499..."
6665,3602.93,0.005368,2.889525,6665.0,6593.0,0.092737,99.0,0.000336,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1832.306673,0.013572,"[3.243470388221461, 3.137221003257709, 2.80499..."


katsuura
{"parentScheme": tournament_parent, "recombinationScheme": simple_arithmetic, "nr_of_parents": 10 }


,averageAge,averageFitness,averageOfstdevGenomes,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness,stdevGenomes
6661,2539.7,0.00504,1.578025,6661.0,6655.0,0.039672,16.0,0.001004,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1889.659475,0.006657,"[3.4241739235939552, 2.503809580261706, 1.8122..."
6662,2540.7,0.00504,1.578025,6662.0,6656.0,0.039672,17.0,0.001004,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1889.659475,0.006657,"[3.4241739235939552, 2.503809580261706, 1.8122..."
6663,2541.7,0.00504,1.578025,6663.0,6657.0,0.039672,18.0,0.001004,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1889.659475,0.006657,"[3.4241739235939552, 2.503809580261706, 1.8122..."
6664,2542.7,0.00504,1.578025,6664.0,6658.0,0.039672,19.0,0.001004,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1889.659475,0.006657,"[3.4241739235939552, 2.503809580261706, 1.8122..."
6665,2543.7,0.00504,1.578025,6665.0,6659.0,0.039672,20.0,0.001004,100.0,"{""parentScheme"": tournament_parent, ""recombina...",1889.659475,0.006657,"[3.4241739235939552, 2.503809580261706, 1.8122..."


In [79]:
fn_df = pd.DataFrame()
for fn, total_df in total_dfs.items():
    print(fn)
    mean = total_df.groupby(['run_config', 'generation_nr']).mean()
    mean = mean.reset_index()
    max_idx = mean['maxFitness'].idxmax()
    
#     display(mean.loc[max_idx])
#     df = df.append(obj, ignore_index=True)
    
    row = mean.loc[max_idx]
    row = row.copy()
    print(row.keys())
    row['function'] = fn
    fn_df = fn_df.append(row, ignore_index=True)

# display(fn_df)
display(HTML(fn_df.to_html()))

cigar
Index(['run_config', 'generation_nr', 'averageAge', 'averageFitness',
       'averageOfstdevGenomes', 'maxAge', 'maxFitness', 'minAge', 'minFitness',
       'populationSize', 'stdevAge', 'stdevFitness'],
      dtype='object')
schaffer
Index(['run_config', 'generation_nr', 'averageAge', 'averageFitness',
       'averageOfstdevGenomes', 'maxAge', 'maxFitness', 'minAge', 'minFitness',
       'populationSize', 'stdevAge', 'stdevFitness'],
      dtype='object')
katsuura
Index(['run_config', 'generation_nr', 'averageAge', 'averageFitness',
       'averageOfstdevGenomes', 'maxAge', 'maxFitness', 'minAge', 'minFitness',
       'populationSize', 'stdevAge', 'stdevFitness'],
      dtype='object')


,averageAge,averageFitness,averageOfstdevGenomes,function,generation_nr,maxAge,maxFitness,minAge,minFitness,populationSize,run_config,stdevAge,stdevFitness
0,12.77,2.198684,0.640734,cigar,71.0,47.0,6.934916,0.0,9.875560e-01,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",10.349739,1.139977
1,11.98,2.605943,0.140045,schaffer,99.0,46.0,3.591583,0.0,2.124162e+00,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",9.037677,0.356277
2,43.75,0.000018,2.862446,katsuura,75.0,76.0,0.001820,0.0,4.483800e-16,100.0,"{""parentScheme"": uniform, ""recombinationScheme...",23.095616,0.000181


In [72]:

def plot_columns(df):    
    for col in df:
        print(col)
        if col not in {'averageAge', 'averageFitness', 'maxFitness', 'averageFitness'}:
            continue

        plt.figure(figsize=(10, 5))
        sns.scatterplot(y=df[col], x=range(len(df[col])))

        plt.show()
        

def plot_columns_parents(df):    
    for col in df:
        print(col)
        if col not in {'averageAge', 'averageFitness', 'maxFitness', 'averageFitness'}:
            continue

        
        plt.figure(figsize=(10, 5))
#             sns.scatterplot(y=df[col], x=range(len(df[col])), hue=df['parents'])
        sns.scatterplot(y=df[col], x=range(len(df[col])), hue=df['parents'])

        plt.show()


for fn, total_df in total_dfs.items():
    total_df = total_df.copy()
    print(fn)
    
    total_df['parents'] = total_df['run_config'].map(lambda x: int(re.search('nr_of_parents\": ([0-9]+)', x).groups(1)[0]))
    new_col = total_df['run_config'].map(lambda x: (re.search('(.+) \"nr_of_parents\": ([0-9]+)', x).groups(1)[0]))
    total_df = total_df.assign(run_config_wo_parents=new_col.values)
#     total_df['run_config_wo_parents'] = 
#     display(total_df)
#     return
#     print(total_df['run_config_without_parents'])
    print(total_df.keys())
    
    mean = total_df.groupby(['run_config', 'generation_nr']).mean()
#     display(mean)
    print(mean.keys())
    mean = mean.reset_index()
    print(mean.keys())
    
    for name, group in mean.groupby('run_config_wo_parents'):
        print(name)
        display(group)
        plot_columns_parents(group)

cigar
Index(['averageAge', 'averageFitness', 'averageOfstdevGenomes',
       'generation_nr', 'maxAge', 'maxFitness', 'minAge', 'minFitness',
       'populationSize', 'run_config', 'stdevAge', 'stdevFitness',
       'stdevGenomes', 'parents', 'run_config_without_parents',
       'run_config_wo_parents'],
      dtype='object')
Index(['averageAge', 'averageFitness', 'averageOfstdevGenomes', 'maxAge',
       'maxFitness', 'minAge', 'minFitness', 'populationSize', 'stdevAge',
       'stdevFitness', 'parents'],
      dtype='object')
Index(['run_config', 'generation_nr', 'averageAge', 'averageFitness',
       'averageOfstdevGenomes', 'maxAge', 'maxFitness', 'minAge', 'minFitness',
       'populationSize', 'stdevAge', 'stdevFitness', 'parents'],
      dtype='object')


KeyError: 'run_config_wo_parents'

In [ ]:
# sns.pairplot(df)

# plt.show()